In [2]:
from tensorflow import keras
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import GlobalMaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten

from keras.layers.core import Dense, Dropout, Activation, Flatten
import numpy as np
# from keras.utils.np_utils import to_categorical
# import matplotlib.pyplot as plt
from keras.applications.mobilenet_v2 import MobileNetV2
from sklearn.model_selection import train_test_split

In [3]:
data_train = np.load('Dataset5_raw_train.npz')
image_train, image_label_train = data_train['image'].astype(np.float16) / 255.0, data_train['image_label'].astype(
    np.int8)

In [4]:
x_train = np.concatenate([image_train[:2326], image_train[2512:4838], image_train[6119:8445], image_train[12234:14560],
                          image_train[16672:18998]])
y_train = np.concatenate([image_label_train[:2326], image_label_train[2512:4838], image_label_train[6119:8445],
                          image_label_train[12234:14560], image_label_train[16672:18998]])

In [5]:
data_val = np.load('Dataset5_raw_val.npz')
image_val, image_label_val = data_val['image'].astype(np.float16) / 255.0, data_val['image_label'].astype(np.int8)

x_val = np.concatenate(
    [image_val[:836], image_val[837:1673], image_val[2040:2876], image_val[4079:4915], image_val[5559:6395]])
y_val = np.concatenate(
    [image_label_val[:836], image_label_val[837:1673], image_label_val[2040:2876], image_label_val[4079:4915],
     image_label_val[5559:6395]])

In [6]:
IMG_SHAPE = (224, 224, 3)
# Pre-trained model with MobileNetV2
base_model = MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)
# Freeze the pre-trained model weights
base_model.trainable = True

for layer in base_model.layers[:100]:
    layer.trainable = False

# Trainable classification head
maxpool_layer = GlobalMaxPooling2D()
flatten = Flatten()
dense1 = Dense(1024, activation='relu')
prediction_layer = Dense(units=5, activation='softmax')
# Layer classification head with feature detector
model = Sequential([
    base_model,
    maxpool_layer, flatten, dense1,
    prediction_layer
])

trainX = x_train
testX = x_val
trainY = y_train
testY = y_val

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="Adam", metrics=["accuracy"])

H = model.fit(trainX, trainY, batch_size=16, steps_per_epoch=len(trainX) // 32, validation_data=(testX, testY),
              validation_steps=len(testX) // 32, epochs=20)

Epoch 1/20
363/363 [==============================] - 81s 101ms/step - loss: 3.1415 - accuracy: 0.5971 - val_loss: 3.6438 - val_accuracy: 0.1957
Epoch 2/20
363/363 [==============================] - 24s 67ms/step - loss: 0.4913 - accuracy: 0.8370 - val_loss: 2.2538 - val_accuracy: 0.4216
Epoch 3/20
363/363 [==============================] - 25s 67ms/step - loss: 0.3616 - accuracy: 0.8727 - val_loss: 0.5239 - val_accuracy: 0.8091
Epoch 4/20
363/363 [==============================] - 24s 67ms/step - loss: 0.3310 - accuracy: 0.8887 - val_loss: 0.3436 - val_accuracy: 0.8909
Epoch 5/20
363/363 [==============================] - 24s 67ms/step - loss: 0.2727 - accuracy: 0.9081 - val_loss: 0.4823 - val_accuracy: 0.8611
Epoch 6/20
363/363 [==============================] - 24s 67ms/step - loss: 0.2648 - accuracy: 0.9136 - val_loss: 0.3018 - val_accuracy: 0.8938
Epoch 7/20
363/363 [==============================] - 24s 67ms/step - loss: 0.2660 - accuracy: 0.9096 - val_loss: 1.2238 - val_accuracy